# Preprocesamiento

## Importar librerias

### Modelo inicial sin GridSearch

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from  xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
df = pd.read_excel("Tabla Trabajo Grupal N°2.xlsx", sheet_name= "Desarrollo")
df.head()

,Id_Cliente,Edad,Nivel_Educacional,Años_Trabajando,Ingresos,Deuda_Comercial,Deuda_Credito,Otras_Deudas,Ratio_Ingresos_Deudas,Default
0,1,34,SupInc,11,68,11.9,3.37,4.73,0.29,1
1,2,43,Med,3,53,9.8,1.51,3.69,0.28,0
2,3,36,SupInc,7,118,11.2,8.48,4.73,0.21,0
3,4,46,Bas,11,61,10.4,3.44,2.90,0.27,1
4,5,29,Med,0,21,12.4,0.38,2.22,0.71,0


In [ ]:
def limpiar_columnas(df):
    df.drop("Id_Cliente",  errors= "ignore", inplace = True)
    return df.drop("Default", axis = 1), df["Default"]

df_x, df_y = limpiar_columnas(df)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state= 7, test_size=0.3)
var_num = df_x.select_dtypes("number").columns
var_cat = df_x.select_dtypes("object").columns
pip_cat = Pipeline(steps = [
                            ("ohe", OneHotEncoder())
                            ]
                    )

pp = ColumnTransformer(transformers= 
                       [("ohe", pip_cat, var_cat)],remainder='passthrough')

model_pipe = Pipeline(steps =
                      [
                        ("pp", pp)
                       ,("RL", LogisticRegression())
                          
                      ]
                    )

#model_pipe.fit(x_train, y_train)


El accuracy del modelo es: 0.755867278122471


c:\Users\hemer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Modelo con GridSearch

In [22]:
from sklearn.model_selection import GridSearchCV

In [32]:
param_grid = {
    "RL__C": [0.1, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
    "RL__penalty": ["l1", "l2"],
    "RL__solver": ["liblinear"]
}

busqueda = GridSearchCV(
    estimator = model_pipe,
    param_grid= param_grid,
    cv = 5,
    n_jobs= -1,
    scoring= "accuracy",
    
)

busqueda.fit(x_train, y_train)
y_predict = model_pipe.predict(x_test)

acc = accuracy_score(y_test, y_predict)

print(f"El accuracy del modelo es: {acc}")
print(f"Los mejores parametros fueron {busqueda.best_params_}")

El accuracy del modelo es: 0.755867278122471
Los mejores parametros fueron {'RL__C': 5, 'RL__penalty': 'l1', 'RL__solver': 'liblinear'}


In [34]:
param_grid = [
    # COMPETIDOR 1: Regresión Logística
    {
        'RL': [LogisticRegression(max_iter=1000)],
        'RL__C': [0.1, 1, 10],
        'RL__solver': ['liblinear']
    },
    # COMPETIDOR 2: Random Forest
    {
        'modelo': [RandomForestClassifier(random_state=7)],
        'modelo__n_estimators': [100, 200],
        'modelo__max_depth': [None, 10, 20]
    },
    # COMPETIDOR 3: XGBoost
    {
        'modelo': [XGBClassifier(random_state=7, use_label_encoder=False, eval_metric='logloss')],
        'modelo__learning_rate': [0.01, 0.1],
        'modelo__n_estimators': [100, 200]
    }
]

busqueda_competitiva = GridSearchCV(
    estimator=model_pipe,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

busqueda_competitiva.fit(x_train, y_train)

# --- RESULTADOS ---
print(f"El modelo ganador es: {busqueda_competitiva.best_params_['modelo']}")
print(f"Mejor Accuracy (CV): {busqueda_competitiva.best_score_:.4f}")



Fitting 5 folds for each of 13 candidates, totalling 65 fits


ValueError: Invalid parameter 'modelo' for estimator Pipeline(steps=[('pp',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder())]),
                                                  Index(['Nivel_Educacional'], dtype='object'))])),
                ('RL', LogisticRegression())]). Valid parameters are: ['memory', 'steps', 'transform_input', 'verbose'].